Lab 2 - Data Reading and Processing
Name: Michael

In [2]:
#importing needed functions
import os
import re 
import json
import pandas as pd 
import numpy as np 

In [3]:
#clean numeric text to floats
#handling commas, parentheses for negatives
def parse_money(x):
    """converting various strings to floats which returns np.nan when failed"""
    if pd.isna(x):
        return np.nan
    #convert to string and strip the whitespace 
    s = str(x).strip()
    if s == '' or s.upper() in {"N.A.", "NA", "N/A", "Null"}:
        return np.nan
#treating the parentheses as negative numbers
#removing stray characters
#replacing missreads
    s = s.replace('$','') 
    s = s.replace(',','') 
    s = s.replace('\u2009','') 

    if re.search(r"\d", s) and re.search(r"[Oo]", s):
        s = re.sub(r"[Oo]", '0', s)
#handleing parentheses 
    neg = False
    if s.startswith('(') and s.endswith(')'):
        neg = True
        s = s[1:-1]
#removing any remaming non numerics & non dots & non minus characters
    s = re.sub(r"[^0-9.\-]", '', s)
    if s in ['', '.', '-', '-.']:
        return np.nan
    try:
        v = float(s)
        return -v if neg else v
    except Exception:
        return np.nan

In [4]:
#read fortune500.csv
fortune_path = 'fortune500.csv'
if os.path.exists(fortune_path):
#trying to read with pandas allowing bad lines
    try:
        df_csv = pd.read_csv(fortune_path, engine='python', on_bad_lines='skip', dtype=str)
    except Exception:
#reading line-by-line and attempting a manual split on commas while saving quoted parts
        lines = open(fortune_path, 'r', encoding='utf-8', errors='ignore').read().splitlines()
        parsed = []
        for line in lines:
#naive split if JSON-like line, try to parse as JSON
            parsed.append(line.split(','))
        df_csv = pd.DataFrame(parsed[1:], columns=parsed[0])
else:
    df_csv = pd.DataFrame() # empty placeholder if file missing

In [5]:
#read lines.json
lines_path = 'lines.json'
#will hold parsed json objects
json_rows = []
if os.path.exists(lines_path):
    with open(lines_path, 'r', encoding='utf-8', errors='ignore') as f:
        for raw in f:
            raw = raw.strip()
            if not raw:
                continue
            try:
#many files are like JSON objects per line
                obj = json.loads(raw)
                json_rows.append(obj)
            except Exception:
#trying to salvage by replacing trailing commas or fixing common mistakes
                try:
#remove trailing commas before the closing brace
                    cleaned = re.sub(r',\s*}', '}', raw)
                    obj = json.loads(cleaned)
                    json_rows.append(obj)
                except Exception:
#giving up on this line
                    continue
    df_json = pd.DataFrame(json_rows)
else:
    df_json = pd.DataFrame()

In [6]:
#read unstructureddata (1).txt
unstructured_path = 'unstructureddata (1).txt'
unstructured_rows = []
if os.path.exists(unstructured_path):
    text = open(unstructured_path, 'r', encoding='utf-8', errors='ignore').read()
#trying to extract JSON-like or CSV-like lines
#lines containing Year and Company and Revenue and Profit keys
    candidate_lines = re.findall(r"\{[^\}]*\}", text) # find {...} blocks
    if candidate_lines:
        for block in candidate_lines:
            try:
                obj = json.loads(block)
                unstructured_rows.append(obj)
            except Exception:
#cleanup and try a trivial key - value extraction
                kvs = re.findall(r'"?([A-Za-z ]+)"?\s*:\s*"?([^,\}\n]+)"?', block)
                d = {k.strip(): v.strip() for k, v in kvs}
            if d:
                unstructured_rows.append(d)
    else:
#each line might be a CSV or whitespace separated. Try splitting by newline and colon/comma
        for line in text.splitlines():
            line = line.strip()
            if not line:
                continue
#try to split on comma into four or five parts
            parts = [p.strip() for p in re.split(r',\s*(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)', line) if p.strip()]
            if len(parts) >= 3:
#heuristically assign the fields
#ex: Year, Rank, Company, Revenue, Profit
                d = {}
                if re.match(r'^\d{4}$', parts[0]):
                    d['Year'] = parts[0]
                    if len(parts) >= 5:
                        d['Rank'] = parts[1]
                        d['Company'] = parts[2]
                        d['Revenue (in millions)'] = parts[3]
                        d['Profit (in millions)'] = parts[4]
                    elif len(parts) == 4:
                        d['Company'] = parts[1]
                        d['Revenue (in millions)'] = parts[2]
                        d['Profit (in millions)'] = parts[3]
                else:
# try to find a year anywhere
                    y = re.search(r'(19|20)\d{2}', line)
                    if y:
                        d['Year'] = y.group(0)
#attempt to pull numbers for revenue/profit from end of line
                        nums = re.findall(r'[-()\d,\.O]+', line)
                        if len(nums) >= 2:
                            d['Revenue (in millions)'] = nums[-2]
                            d['Profit (in millions)'] = nums[-1]
#company name - substring between year and first number
                        m = re.search(rf"{d['Year']}\D+(.+?)\s+{re.escape(nums[-2])}", line) if nums else None
                        if m:
                            d['Company'] = m.group(1).strip()
                    if d:
                        unstructured_rows.append(d)
    df_unstructured = pd.DataFrame(unstructured_rows)
else:
    df_unstructured = pd.DataFrame()

In [7]:
#normalize and standardize column names and union the dataframes
#helper to standardize a dataframe's columns to- Year, Rank, Company, Revenue, Profit
def standardize_df(df):
    """Map common variants of column names to a chosen canonical set and return a cleaned copy."""
    if df.empty:
        return df
    df = df.copy()
#lower-case columns for mapping
    colmap = {}
    for c in df.columns:
        lc = c.strip().lower()
        if 'year' in lc:
            colmap[c] = 'Year'
        elif 'rank' in lc:
            colmap[c] = 'Rank'
        elif 'company' in lc:
            colmap[c] = 'Company'
        elif 'revenue' in lc:
            colmap[c] = 'Revenue'
        elif 'profit' in lc:
            colmap[c] = 'Profit'
        else:
#save unknown columns as-is
            colmap[c] = c
    df.rename(columns=colmap, inplace=True)
#ensure columns exist
    for needed in ['Year', 'Rank', 'Company', 'Revenue', 'Profit']:
        if needed not in df.columns:
            df[needed] = np.nan
#strip whitespace from the company names
    df['Company'] = df['Company'].astype(str).str.strip().replace({'nan': np.nan})
#clean revenue and profit columns using parse_money
    df['Revenue'] = df['Revenue'].apply(parse_money)
    df['Profit'] = df['Profit'].apply(parse_money)
#year and Rank to numeric when possible
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
    df['Rank'] = pd.to_numeric(df['Rank'], errors='coerce')
    return df[['Year', 'Rank', 'Company', 'Revenue', 'Profit']]


#standardize available dataframes
std_csv = standardize_df(df_csv)
std_json = standardize_df(df_json)
std_un = standardize_df(df_unstructured)


#combine them vertically
combined = pd.concat([std_csv, std_json, std_un], ignore_index=True, sort=False)


#drop exact duplicate rows
combined_before = combined.shape[0]
combined.drop_duplicates(inplace=True)
combined_after = combined.shape[0]

In [8]:
#further cleaning and standardization
#remove rows that have neither company nor year
combined = combined[~(combined['Company'].isna() & combined['Year'].isna())].copy()


#trim company names and normalize some obvious variants
combined['Company'] = combined['Company'].astype(str).str.strip()
#more normalizaiton
company_normalization = {
    'exxon mobil': 'Exxon Mobil',
    'chevrontexaco': 'ChevronTexaco',
    'intl. business machines': 'International Business Machines',
}
combined['Company_norm'] = combined['Company'].str.lower().map(lambda s: company_normalization.get(s, None))
combined['Company'] = combined.apply(lambda r: r['Company_norm'] if pd.notna(r['Company_norm']) else r['Company'], axis=1)
combined.drop(columns=['Company_norm'], inplace=True)


#remove rows where revenue and profit are both NaN and Company is NaN
combined = combined[~(combined['Company'].isna() & combined['Revenue'].isna() & combined['Profit'].isna())]


#save the cleaned combined dataset to CSV
combined.to_csv('processed_combined.csv', index=False)

In [9]:
#good data - rows where Year, Company, Revenue, Profit are all present (non-NaN)
good_mask = combined['Year'].notna() & combined['Company'].notna() & combined['Revenue'].notna() & combined['Profit'].notna()
good_data_count = int(good_mask.sum())
#bad data - rows where any of the needed fields missing
bad_data_count = int((~good_mask).sum())
#number of unique companies based on the non-null company names
unique_companies = int(combined['Company'].dropna().nunique())

#determining the company with highest revenue from 1995 to 1998
start_year = 1995
end_year = 1998
#check available years
available_years = combined['Year'].dropna().unique()
if any((available_years >= start_year) & (available_years <= end_year)):
#filter to the target range
    sub = combined[(combined['Year'] >= start_year) & (combined['Year'] <= end_year)].copy()
#group by company and sum revenue & profit across the years
    grp = sub.groupby('Company').agg({'Revenue':'sum','Profit':'sum'}).reset_index()
    if not grp.empty:
        top_rev_row = grp.loc[grp['Revenue'].idxmax()]
        top_profit_row = grp.loc[grp['Profit'].idxmax()]
        top_revenue_company = top_rev_row['Company']
        top_revenue_value = float(top_rev_row['Revenue'])
        top_profit_company = top_profit_row['Company']
        top_profit_value = float(top_profit_row['Profit'])
    else:
        top_revenue_company = 'No data in 1995-1998'
        top_revenue_value = np.nan
        top_profit_company = 'No data in 1995-1998'
        top_profit_value = np.nan
    year_note = f"Used range {start_year}-{end_year}."
else:
#use the available year range and compute top companies across all available years
    min_y = int(np.nanmin(available_years)) if len(available_years)>0 else None
    max_y = int(np.nanmax(available_years)) if len(available_years)>0 else None
    if min_y is None:
        top_revenue_company = 'No year data available'
        top_revenue_value = np.nan
        top_profit_company = 'No year data available'
        top_profit_value = np.nan
        year_note = 'No years available in dataset.'
    else:
        sub = combined[(combined['Year'] >= min_y) & (combined['Year'] <= max_y)].copy()
        grp = sub.groupby('Company').agg({'Revenue':'sum','Profit':'sum'}).reset_index()
        if not grp.empty:
            top_rev_row = grp.loc[grp['Revenue'].idxmax()]
            top_profit_row = grp.loc[grp['Profit'].idxmax()]
            top_revenue_company = top_rev_row['Company']
            top_revenue_value = float(top_rev_row['Revenue'])
            top_profit_company = top_profit_row['Company']
            top_profit_value = float(top_profit_row['Profit'])
        else:
            top_revenue_company = 'No revenue data'
            top_revenue_value = np.nan
            top_profit_company = 'No profit data'
            top_profit_value = np.nan
            year_note = f'1995-1998 not present; used available range {min_y}-{max_y}.'

#Results_Combine dataframe
results = {
    'Metric': [
        'Total Good Rows (complete Year, Company, Revenue, Profit)',
        'Total Bad Rows (incomplete)',
        'Unique Companies (non-null names)',
        'Top Revenue Company (1995-1998)',
        'Top Revenue Value (in millions)',
        'Top Profit Company (1995-1998)',
        'Top Profit Value (in millions)',
        'Year Note'
    ],
    'Value': [
        good_data_count,
        bad_data_count,
        unique_companies,
        top_revenue_company,
        top_revenue_value,
        top_profit_company,
        top_profit_value,
        year_note
    ]
}
Results_Combine = pd.DataFrame(results).set_index('Metric')
    

In [10]:
#print Results_Combine
print(Results_Combine)

                                                                    Value
Metric                                                                   
Total Good Rows (complete Year, Company, Revenu...                  29011
Total Bad Rows (incomplete)                                           393
Unique Companies (non-null names)                                    2599
Top Revenue Company (1995-1998)                            General Motors
Top Revenue Value (in millions)                                  670322.8
Top Profit Company (1995-1998)                                Exxon Mobil
Top Profit Value (in millions)                                    27540.0
Year Note                                           Used range 1995-1998.


In [23]:
#save Results_Combine to CSV
Results_Combine.to_csv('Results_Combine.csv')